# Client Manager Adminstrator
#### DynamoDB Table - tps-&lt;teamName&gt;-CustomerConfig-&lt;env&gt;

## Client grouping concept
### This notebook is used for onboarding clients for each sdlf team space
### Each Client should be configured in the same AWS Region as the AMC INSTANCE.
### Each client is defined by 
#### 1. AMC instance
#### 2. Corresponding profiles ids which are grouped for this customer according the advertiserids used for setting up the AMC instance

#### -----------------------

## Global Configs
#### Team name as per the foundation installation

#### -----------------------

In [ ]:
## The Team Name configured in the SDLF platform
SDLF_TEAM_NAME = "demoteam"
## ARN of the secrets manager which is used to store the advertising api credentials. This has to created manually due to security compliance. 
## It should be created in the same region as the AMC instance
SAS_SECRET_MANAGER_ARN = "secret manger arn"
## Region of the AMC instance
aws_region = "us-east-1"

In [ ]:
import boto3
if len(aws_region) == 0:
    aws_region = str(boto3.Session().region_name)
print("Region : " + aws_region)

## Sponsored Ads Base URL configuration
### This section might need to be updated by referring to the https://advertising.amazon.com/API/docs/en-us/get-started/how-to-use-api > API Endpoints

In [ ]:
import json

sas_base_api_url = {
    "us-east-1" : "https://advertising-api.amazon.com",
    "eu-west-1" : "https://advertising-api-eu.amazon.com",
    "me-south-1" : "https://advertising-api-fe.amazon.com"
}

print(json.dumps(sas_base_api_url, indent=4))

## To view the list of configured client details
#### Run the below cell to view the list

#### -----------------------

In [ ]:
from atsclientslibraries.clientmgr import customer
import pandas as pd
## Pandas display configs
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_rows', None)

df = customer.get_customers_config(ATS_TEAM_NAME=SDLF_TEAM_NAME)
df.head()

#### -----------------------

## To Insert or Update client details
####        Run the below cell to setup abd verify the client configuration
```
{
    "customer_id": <Customer ID which will be used to tag resources like bucket prefixes and table names in the platform. It has to unique and can be the name of the amc instance>,
    
    "customer_name":<Customer Name. This will be used to idenify a customer. Keep it less than 5 characters and avoid any special characters>,
    
    "customer_type": <ENDEMIC or NON-ENDEMIC>,
    
    "region":<Region of the AMC instance. This can be requested or retrieved from the ticket raised to the AMC team>,
    
    "amc":{
        "sdlf_datset_name":<Name of the dataset which is configured in SDLF to process the AMC datasets>,
        
        "endpoint_url":<AMC instance endpoint. This can be requested or retrieved from the ticket raised to the AMC team >,
        
        "aws_orange_account_id":<AMC Orange Account. This can be requested or retrieved from the ticket raised to the AMC team>,
        
        "aws_green_account_id":<AMC Green Account. This can be requested or retrieved from the ticket raised to the AMC team>,
        
        "bucket_name":<Name of the bucket for the AMC Dataset landing bucket in your AWS account. This can be requested or retrieved from the ticket raised to the AMC team>,
        
        "advertiser_ids":<Comma seperated list of advertiser ids. This can be retrieved from the ticket raised to the AMC team. It is the list provided while raising the AMC instance>,
        
        "entity_ids":<Comma seperated list of Entity ids. This can be retrieved from the ticket raised to the AMC team. It is the list provided while raising the AMC instance>"
    }  ,  
    
    "sas":{
        
        "sdlf_datset_name":<Name of the dataset which is configured in SDLF to process the AMC datasets>,
        
        "credential_arn":<ARN of the secrets manager which is used to store the advertising api credentials. This has to created manually due to security compliance>,
        
        "profile_details":<Comma seperated Profile ids for which Sponsored Products/Brands/displayADs/DSP related reports/metadata are retrieved. It can be set to "ALL" if reports/metadata needs to retrieved for all profile ids. The list of profile IDs can be found on Advertising API console. You can also use the "Retrieve Profiles" section to get a list of >
        
        "api_base_url":<This is the base url used for specific region. In the "Sponsored Ads Base URL configuration" section has details about region specific API's base urls. This section might need to be updated by referring to the https://advertising.amazon.com/API/docs/en-us/get-started/how-to-use-api > API Endpoints >
    }
}
```

In [ ]:
import json

customer_details = {
    "customer_id":"democustomer",
    "customer_name":"DemoCustomer",
    "customer_type": "ENDEMIC",
    "region":aws_region,
    "amc":{
        "sdlf_datset_name":"amcdataset",
        "endpoint_url":"<amc url>",
        "aws_orange_account_id":"<amc orange account number>",
        "aws_green_account_id":"<amc green account number>",
        "bucket_name":"<amc bucketname>",
        "advertiser_ids":"<comma seperated advertiser ids>",
        "entity_ids":"<comma seperated entity ids>"
    }
}

print(json.dumps(customer_details, indent=4))

### Run the below cell to submit the client configuration. 
#### Before submitting verify the configuration by running the above cell

In [ ]:
from atsclientslibraries.clientmgr import customer

import boto3

dynamodb_resp_wr = customer.set_customers_config(customer_details=customer_details, ATS_TEAM_NAME=SDLF_TEAM_NAME)
dynamodb_resp_wr